# Casys MCP Sandbox - Basic Usage

This notebook demonstrates the core capabilities of the Casys MCP Gateway sandbox:

- Secure code execution in isolated Deno subprocess
- Result handling and serialization
- Error capture and reporting
- Timeout enforcement

**Documentation:** [JSR Package](https://jsr.io/@casys/mcp-gateway)

## 1. Import Sandbox from JSR

The sandbox is published on JSR and can be imported directly:

In [1]:
import { DenoSandboxExecutor } from "jsr:@casys/mcp-gateway";

// Create sandbox instance with default config
const sandbox = new DenoSandboxExecutor({
  timeout: 5000, // 5s max execution time
  memoryLimit: 128, // 128MB max memory
});

console.log("✅ Sandbox initialized");

✅ Sandbox initialized


## 2. Simple Execution

Execute basic TypeScript expressions:

In [2]:
const result = await sandbox.execute("return 1 + 1");

console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": 2,
  "executionTimeMs": 56.40794199999982
}


## 3. Data Processing

Execute code with array operations and data transformations:

In [3]:
const code = `
const data = [1, 2, 3, 4, 5];
return {
  sum: data.reduce((a, b) => a + b, 0),
  avg: data.reduce((a, b) => a + b, 0) / data.length,
  max: Math.max(...data),
  doubled: data.map(x => x * 2)
};
`;

const result = await sandbox.execute(code);
console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": {
    "sum": 15,
    "avg": 3,
    "max": 5,
    "doubled": [
      2,
      4,
      6,
      8,
      10
    ]
  },
  "executionTimeMs": 55.886519000000135
}


## 4. Syntax Error Handling

The sandbox captures syntax errors gracefully:

In [4]:
const result = await sandbox.execute("return 1 + + 1"); // Invalid syntax

console.log(JSON.stringify(result, null, 2));

INFO Sandbox execution succeeded
{
  "success": true,
  "result": 2,
  "executionTimeMs": 53.12775199999987
}


## 5. Runtime Error Handling

Runtime errors are also captured and returned:

In [5]:
const code = `
const obj = undefined;
return obj.property; // Will throw TypeError
`;

const result = await sandbox.execute(code);
console.log(JSON.stringify(result, null, 2));

{
  "success": false,
  "error": {
    "type": "RuntimeError",
    "message": "Cannot read properties of undefined (reading 'property')",
    "stack": "TypeError: Cannot read properties of undefined (reading 'property')\n    at file:<temp-file>:8:12\n    at file:<temp-file>:10:6\n    at file:<temp-file>:33:2"
  },
  "executionTimeMs": 53.6858059999995
}


## 6. Timeout Enforcement

Long-running code is automatically terminated after the timeout:

In [6]:
const code = `
// Infinite loop - will be killed after 5 seconds
while (true) {
  // This will timeout
}
`;

const result = await sandbox.execute(code);
console.log(JSON.stringify(result, null, 2));

WARN Sandbox execution timeout, killing process
{
  "success": false,
  "error": {
    "type": "TimeoutError",
    "message": "Execution exceeded timeout of 5000ms"
  },
  "executionTimeMs": 5008.977321
}


## Summary

The Casys MCP Sandbox provides:

- ✅ **Isolated execution** - Code runs in separate Deno subprocess
- ✅ **Error handling** - Syntax and runtime errors are captured
- ✅ **Timeout protection** - Prevents infinite loops
- ✅ **Result serialization** - Structured JSON output
- ✅ **Memory limits** - Prevents resource exhaustion

**Next Steps:**

- Try `context-injection.ipynb` for advanced context passing
- See `security-demo.ipynb` for security features
- Read the [full documentation](https://jsr.io/@casys/mcp-gateway)